In [27]:
!pip install dash
!pip install dash-leaflet

In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign
import ipywidgets as widgets

In [5]:
######################_____DV_7____######################

# Funzioni per ottenere le opzioni

def get_pollutants():
    t = requests.get("http://127.0.0.1:5000/api/pollutants")
    return [{'label': p, 'value': p} for p in read_response(t)]

def get_municipalities():
    t = requests.get("http://127.0.0.1:5000/api/municipalities")
    return [{'label': m, 'value': m} for m in read_response(t)]

def get_provinces():
    t = requests.get("http://127.0.0.1:5000/api/provinces")
    return [{'label': p, 'value': p} for p in read_response(t)]

# Avvio app Dash
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H2("DV_7: Average pollutant value by municipality or province"),

    html.Div([
        html.Label("Select type (Comune or Provincia):"),
        dcc.RadioItems(
            id='tipo-radio',
            options=[{'label': 'Comune', 'value': 'Comune'}, {'label': 'Provincia', 'value': 'Provincia'}],
            value='Comune',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ]),

    html.Div([
        html.Label("Select name:"),
        dcc.Dropdown(id='nome-dropdown')
    ], style={'margin-top': '20px'}),

    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown', options=get_pollutants())
    ], style={'margin-top': '20px'}),

    html.Button("Visualize time series", id='submit-button', n_clicks=0, style={'margin-top': '20px'}),

    dcc.Loading(
        id="loading",
        children=dcc.Graph(id='time-series-graph'),
        type="dot"
    )
])

# Callback per aggiornare i nomi
@app.callback(
    Output('nome-dropdown', 'options'),
    Input('tipo-radio', 'value')
)
def update_names(tipo):
    return get_municipalities() if tipo == 'Comune' else get_provinces()

# Callback per ottenere e visualizzare i dati
@app.callback(
    Output('time-series-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    State('tipo-radio', 'value'),
    State('nome-dropdown', 'value'),
    State('pollutant-dropdown', 'value')
)
def update_graph(n_clicks, tipo, nome, pollutant):
    if n_clicks == 0 or not nome or not pollutant:
        return px.line(title="Please select all inputs and press the button.")

    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)

    if not data:
        return px.line(title=f"No data found for {tipo} '{nome}' and pollutant '{pollutant}'.")

    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'])
    df = df.sort_values('data')

    fig = px.line(df, x='data', y='valore', markers=True,
                  title=f"Time series of {pollutant} in {tipo.lower()} {nome}",
                  labels={'valore': 'Average value', 'data': 'Date'})
    return fig

# Run server
if __name__ == '__main__':
    app.run(debug=True)



In [6]:
import dash
from dash import dcc, html, Output, Input, State
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import plotly.express as px
import plotly.graph_objects as go

# -----------------------------
# Utility per leggere la risposta JSON
# -----------------------------
def read_response(response):
    if response.status_code == 200:
        return response.json()
    return None

# -----------------------------
# API CALLS
# -----------------------------
def get_pollutants():
    r = requests.get("http://127.0.0.1:5000/api/pollutants")
    return [{'label': p, 'value': p} for p in read_response(r) or []]

def get_municipalities():
    r = requests.get("http://127.0.0.1:5000/api/municipalities")
    return [{'label': m, 'value': m} for m in read_response(r) or []]

def get_provinces():
    r = requests.get("http://127.0.0.1:5000/api/provinces")
    return [{'label': p, 'value': p} for p in read_response(r) or []]

def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {
        "pollutant": pollutant,
        "start_date": start_date,
        "end_date": end_date
    }
    r = requests.post(url, json=payload)
    return pd.DataFrame(read_response(r) or [])

def get_all_province_shapes():
    url = "http://127.0.0.1:5000/api/province_shape_v2"
    r = requests.get(url)
    data = read_response(r)
    if data:
        df = pd.DataFrame(data)
        df["geometry"] = df["geometry_province_wkt"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    else:
        return gpd.GeoDataFrame(columns=["nome_provincia", "geometry"], geometry="geometry", crs="EPSG:4326")

# -----------------------------
# App Dash
# -----------------------------
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("DV_7: Time series by municipality or province"),

    html.Div([
        html.Label("Select type:"),
        dcc.RadioItems(
            id='tipo-radio',
            options=[{'label': 'Comune', 'value': 'Comune'}, {'label': 'Provincia', 'value': 'Provincia'}],
            value='Comune',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ]),

    html.Div([
        html.Label("Select name:"),
        dcc.Dropdown(id='nome-dropdown')
    ], style={'margin-top': '10px'}),

    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown', options=get_pollutants())
    ], style={'margin-top': '10px'}),

    html.Button("Visualize time series", id='submit-button', n_clicks=0, style={'margin-top': '10px'}),
    dcc.Loading(id="loading", children=dcc.Graph(id='time-series-graph'), type="dot"),

    html.Hr(),

    html.H2("DV_8: Average concentration by province"),

    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown-dv8', options=get_pollutants())
    ], style={'margin-top': '10px'}),

    html.Div([
        html.Label("Select start date:"),
        dcc.DatePickerSingle(id='start-date-picker', display_format='YYYY-MM-DD')
    ], style={'margin-top': '10px', 'display': 'inline-block', 'margin-right': '20px'}),

    html.Div([
        html.Label("Select end date:"),
        dcc.DatePickerSingle(id='end-date-picker', display_format='YYYY-MM-DD')
    ], style={'margin-top': '10px', 'display': 'inline-block'}),

    html.Button("Show DV_8 results", id='btn-dv8', n_clicks=0, style={'margin-top': '10px'}),

    dcc.Loading(
        id="loading-dv8",
        children=[
            dcc.Graph(id='bar-chart-dv8'),
            dcc.Graph(id='map-dv8', style={'height': '600px'})
        ],
        type="dot"
    )
])

# -----------------------------
# Callback per aggiornare dropdown nomi (DV_7)
# -----------------------------
@app.callback(
    Output('nome-dropdown', 'options'),
    Input('tipo-radio', 'value')
)
def update_names(tipo):
    if tipo == 'Comune':
        return get_municipalities()
    else:
        return get_provinces()

# -----------------------------
# Callback per DV_7 (serie temporale)
# -----------------------------
@app.callback(
    Output('time-series-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    State('tipo-radio', 'value'),
    State('nome-dropdown', 'value'),
    State('pollutant-dropdown', 'value')
)
def update_time_series(n_clicks, tipo, nome, pollutant):
    if n_clicks == 0 or not nome or not pollutant:
        return px.line(title="Please select all inputs and press the button.")

    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)

    if not data:
        return px.line(title=f"No data found for {tipo} '{nome}' and pollutant '{pollutant}'.")

    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'])
    df = df.sort_values('data')

    fig = px.line(df, x='data', y='valore', markers=True,
                  title=f"Time series of {pollutant} in {tipo.lower()} {nome}",
                  labels={'valore': 'Average value', 'data': 'Date'})
    fig.update_layout(template='plotly_dark')
    return fig

# -----------------------------
# Callback per DV_8 (bar chart + map)
# -----------------------------
@app.callback(
    [Output('bar-chart-dv8', 'figure'),
     Output('map-dv8', 'figure')],
    Input('btn-dv8', 'n_clicks'),
    State('pollutant-dropdown-dv8', 'value'),
    State('start-date-picker', 'date'),
    State('end-date-picker', 'date')
)
def update_dv8(n_clicks, pollutant, start_date, end_date):
    if n_clicks == 0 or not pollutant or not start_date or not end_date:
        return {}, {}

    df = get_dv8_data(pollutant, start_date, end_date)
    if df.empty:
        return {}, {}

    provinces_gdf = get_all_province_shapes()

    # Normalizza nomi per il merge
    df['prov_upper'] = df['province'].str.upper().str.strip()
    provinces_gdf['prov_upper'] = provinces_gdf['nome_provincia'].str.upper().str.strip()

    merged = provinces_gdf.merge(df, on='prov_upper', how='left')

    # Colori base: grigio chiaro = 0, blu chiaro = 1, rosso = 2
    merged['color_code'] = 0  # default grigio

    # Provincia più inquinata
    if merged['average_value'].notnull().any():
        max_val = merged['average_value'].max()
        max_prov = merged.loc[merged['average_value'] == max_val, 'nome_provincia'].values[0]
    else:
        max_prov = None

    for idx, row in merged.iterrows():
        if pd.notnull(row['average_value']):
            if row['nome_provincia'] == max_prov:
                merged.at[idx, 'color_code'] = 2  # rosso
            else:
                merged.at[idx, 'color_code'] = 1  # blu chiaro

    # Barre arancioni con linea media rossa
    bar_fig = px.bar(
        df,
        x='province',
        y='average_value',
        title=f"Media {pollutant} per provincia",
        labels={'average_value': f'Concentrazione media di {pollutant}', 'province': 'Provincia'},
        color_discrete_sequence=['orange']
    )
    bar_fig.add_hline(y=df['average_value'].mean(), line_dash='dash', line_color='red',
                      annotation_text=f"Media regionale: {df['average_value'].mean():.2f}",
                      annotation_position="top left")
    bar_fig.update_layout(template='plotly_dark')

    # Mappa con Plotly Choroplethmapbox senza legenda e con colori personalizzati
    geojson = json.loads(merged.to_json())

    colorscale = [
        [0, 'lightgray'],  # 0 grigio chiaro
        [0.5, 'lightblue'],  # 1 azzurro chiaro
        [1, 'red']          # 2 rosso
    ]

    # Normalizzo color_code tra 0 e 1 per colorscale
    z = merged['color_code'].replace({0:0, 1:0.5, 2:1}).values

    map_fig = go.Figure(go.Choroplethmapbox(
        geojson=geojson,
        locations=merged['nome_provincia'],
        z=z,
        featureidkey='properties.nome_provincia',
        colorscale=colorscale,
        showscale=False,
        marker_line_width=1,
        marker_line_color='black',
        zmin=0,
        zmax=1
    ))

    map_fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=7,
        mapbox_center={"lat": 45.5, "lon": 9.0},
        margin={"r":0,"t":30,"l":0,"b":0},
        coloraxis_showscale=False
    )

    return bar_fig, map_fig

# -----------------------------
# Avvio app
# -----------------------------
if __name__ == '__main__':
    app.run(debug=True)



In [12]:
import dash
from dash import dcc, html, Output, Input, State
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import plotly.express as px
import plotly.graph_objects as go

def read_response(response):
    if response.status_code == 200:
        return response.json()
    return None

def get_pollutants():
    r = requests.get("http://127.0.0.1:5000/api/pollutants")
    return [{'label': p, 'value': p} for p in read_response(r) or []]

def get_municipalities():
    r = requests.get("http://127.0.0.1:5000/api/municipalities")
    return [{'label': m, 'value': m} for m in read_response(r) or []]

def get_provinces():
    r = requests.get("http://127.0.0.1:5000/api/provinces")
    return [{'label': p, 'value': p} for p in read_response(r) or []]

def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {"pollutant": pollutant, "start_date": start_date, "end_date": end_date}
    r = requests.post(url, json=payload)
    return pd.DataFrame(read_response(r) or [])

def get_all_province_shapes():
    url = "http://127.0.0.1:5000/api/province_shape_v2"
    r = requests.get(url)
    data = read_response(r)
    if data:
        df = pd.DataFrame(data)
        df["geometry"] = df["geometry_province_wkt"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    else:
        return gpd.GeoDataFrame(columns=["nome_provincia", "geometry"], geometry="geometry", crs="EPSG:4326")

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("DV_7: Time series by municipality or province"),
    html.Div([
        html.Label("Select type:"),
        dcc.RadioItems(id='tipo-radio', options=[{'label': 'Comune', 'value': 'Comune'}, {'label': 'Provincia', 'value': 'Provincia'}], value='Comune', labelStyle={'display': 'inline-block', 'margin-right': '20px'})
    ]),
    html.Div([
        html.Label("Select name:"),
        dcc.Dropdown(id='nome-dropdown')
    ], style={'margin-top': '10px'}),
    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown', options=get_pollutants())
    ], style={'margin-top': '10px'}),
    html.Button("Visualize time series", id='submit-button', n_clicks=0, style={'margin-top': '10px'}),
    dcc.Loading(id="loading", children=dcc.Graph(id='time-series-graph'), type="dot"),

    html.Hr(),

    html.H2("DV_8: Average concentration by province"),
    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown-dv8', options=get_pollutants())
    ], style={'margin-top': '10px'}),
    html.Div([
        html.Label("Select start date:"),
        dcc.DatePickerSingle(id='start-date-picker', display_format='YYYY-MM-DD')
    ], style={'margin-top': '10px', 'display': 'inline-block', 'margin-right': '20px'}),
    html.Div([
        html.Label("Select end date:"),
        dcc.DatePickerSingle(id='end-date-picker', display_format='YYYY-MM-DD')
    ], style={'margin-top': '10px', 'display': 'inline-block'}),
    html.Button("Show DV_8 results", id='btn-dv8', n_clicks=0, style={'margin-top': '10px'}),
    dcc.Loading(id="loading-dv8", children=[dcc.Graph(id='bar-chart-dv8'), dcc.Graph(id='map-dv8', style={'height': '600px'})], type="dot")
])

@app.callback(Output('nome-dropdown', 'options'), Input('tipo-radio', 'value'))
def update_names(tipo):
    return get_municipalities() if tipo == 'Comune' else get_provinces()

@app.callback(Output('time-series-graph', 'figure'), Input('submit-button', 'n_clicks'), State('tipo-radio', 'value'), State('nome-dropdown', 'value'), State('pollutant-dropdown', 'value'))
def update_time_series(n_clicks, tipo, nome, pollutant):
    if n_clicks == 0 or not nome or not pollutant:
        return px.line(title="Please select all inputs and press the button.").update_layout(template='plotly_white')
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {"var_comune": nome if tipo == "Comune" else None, "var_provincia": nome if tipo == "Provincia" else None, "var_pollutant": pollutant}
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)
    if not data:
        return px.line(title=f"No data found for {tipo} '{nome}' and pollutant '{pollutant}'.").update_layout(template='plotly_white')
    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'])
    df = df.sort_values('data')
    fig = px.line(df, x='data', y='valore', markers=True, title=f"Time series of {pollutant} in {tipo.lower()} {nome}", labels={'valore': 'Average value', 'data': 'Date'})
    return fig.update_layout(template='plotly_white')

@app.callback([Output('bar-chart-dv8', 'figure'), Output('map-dv8', 'figure')], Input('btn-dv8', 'n_clicks'), State('pollutant-dropdown-dv8', 'value'), State('start-date-picker', 'date'), State('end-date-picker', 'date'))
def update_dv8(n_clicks, pollutant, start_date, end_date):
    if n_clicks == 0 or not pollutant or not start_date or not end_date:
        return {}, {}
    df = get_dv8_data(pollutant, start_date, end_date)
    if df.empty:
        return {}, {}

    provinces_gdf = get_all_province_shapes()
    df['prov_upper'] = df['province'].str.upper().str.strip()
    provinces_gdf['prov_upper'] = provinces_gdf['nome_provincia'].str.upper().str.strip()
    merged = provinces_gdf.merge(df, on='prov_upper', how='left')

    # Calcola media solo sulle province con dati validi
    media_globale = merged.loc[merged['average_value'].notnull(), 'average_value'].mean()

    # Funzione per assegnare color_code
    def color_group(row):
        if pd.isnull(row['average_value']):
            return 0.0  # no data -> grigio
        elif row['average_value'] < media_globale:
            return 0.25  # sotto media -> azzurro
        else:
            return 0.75  # sopra media -> rosso

    merged['color_code'] = merged.apply(color_group, axis=1)

    # Grafico a barre
    bar_fig = px.bar(df, x='province', y='average_value',
                     title=f"Media {pollutant} per provincia",
                     labels={'average_value': f'Concentrazione media di {pollutant}', 'province': 'Provincia'},
                     color_discrete_sequence=['blue'])
    bar_fig.add_hline(y=media_globale, line_dash='dash', line_color='red',
                      annotation_text=f"Media regionale: {media_globale:.2f}",
                      annotation_position="top left")
    bar_fig.update_layout(template='plotly_white')

    # Preparazione geojson
    geojson = json.loads(merged.to_json())

    # Definizione colorscale personalizzata
    colorscale = [
        [0.0, 'lightgray'],       # no data
        [0.2499, 'lightgray'],
        [0.25, 'lightblue'],      # sotto media
        [0.7499, 'lightblue'],
        [0.75, 'red'],            # sopra media
        [1.0, 'red']
    ]

    # Creazione mappa
    map_fig = go.Figure(go.Choroplethmapbox(
        geojson=geojson,
        locations=merged['nome_provincia'],
        z=merged['color_code'],
        featureidkey='properties.nome_provincia',
        colorscale=colorscale,
        marker_line_width=1,
        marker_line_color='black',
        zmin=0,
        zmax=1,
        hovertext=merged.apply(lambda r: f"{r['nome_provincia']}: {r['average_value'] if pd.notnull(r['average_value']) else 'No data'}", axis=1),
        hoverinfo='text'
    ))

    map_fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=7,
        mapbox_center={"lat": 45.5, "lon": 9.0},
        margin={"r":0,"t":30,"l":0,"b":0},
        coloraxis_showscale=False  # rimuove la legenda colori
    )

    return bar_fig, map_fig

if __name__ == '__main__':
    app.run(debug=True)

In [16]:
import dash
from dash import dcc, html, Output, Input, State
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import plotly.express as px
import plotly.graph_objects as go

def read_response(response):
    if response.status_code == 200:
        return response.json()
    return None

def get_pollutants():
    r = requests.get("http://127.0.0.1:5000/api/pollutants")
    return [{'label': p, 'value': p} for p in read_response(r) or []]

def get_municipalities():
    r = requests.get("http://127.0.0.1:5000/api/municipalities")
    return [{'label': m, 'value': m} for m in read_response(r) or []]

def get_provinces():
    r = requests.get("http://127.0.0.1:5000/api/provinces")
    return [{'label': p, 'value': p} for p in read_response(r) or []]

def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {"pollutant": pollutant, "start_date": start_date, "end_date": end_date}
    r = requests.post(url, json=payload)
    return pd.DataFrame(read_response(r) or [])

def get_all_province_shapes():
    url = "http://127.0.0.1:5000/api/province_shape_v2"
    r = requests.get(url)
    data = read_response(r)
    if data:
        df = pd.DataFrame(data)
        df["geometry"] = df["geometry_province_wkt"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    else:
        return gpd.GeoDataFrame(columns=["nome_provincia", "geometry"], geometry="geometry", crs="EPSG:4326")

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("DV_7: Time series by municipality or province"),
    html.Div([
        html.Label("Select type:"),
        dcc.RadioItems(id='tipo-radio', options=[{'label': 'Comune', 'value': 'Comune'}, {'label': 'Provincia', 'value': 'Provincia'}], value='Comune', labelStyle={'display': 'inline-block', 'margin-right': '20px'})
    ]),
    html.Div([
        html.Label("Select name:"),
        dcc.Dropdown(id='nome-dropdown')
    ], style={'margin-top': '10px'}),
    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown', options=get_pollutants())
    ], style={'margin-top': '10px'}),
    html.Button("Visualize time series", id='submit-button', n_clicks=0, style={'margin-top': '10px'}),
    dcc.Loading(id="loading", children=dcc.Graph(id='time-series-graph'), type="dot"),

    html.Hr(),

    html.H2("DV_8: Average concentration by province"),
    html.Div([
        html.Label("Select pollutant:"),
        dcc.Dropdown(id='pollutant-dropdown-dv8', options=get_pollutants())
    ], style={'margin-top': '10px'}),
    html.Div([
        html.Label("Select start date:"),
        dcc.DatePickerSingle(id='start-date-picker', display_format='YYYY-MM-DD')
    ], style={'margin-top': '10px', 'display': 'inline-block', 'margin-right': '20px'}),
    html.Div([
        html.Label("Select end date:"),
        dcc.DatePickerSingle(id='end-date-picker', display_format='YYYY-MM-DD')
    ], style={'margin-top': '10px', 'display': 'inline-block'}),
    html.Button("Show DV_8 results", id='btn-dv8', n_clicks=0, style={'margin-top': '10px'}),
    dcc.Loading(id="loading-dv8", children=[dcc.Graph(id='bar-chart-dv8'), dcc.Graph(id='map-dv8', style={'height': '600px'})], type="dot")
])

@app.callback(Output('nome-dropdown', 'options'), Input('tipo-radio', 'value'))
def update_names(tipo):
    return get_municipalities() if tipo == 'Comune' else get_provinces()

@app.callback(Output('time-series-graph', 'figure'), Input('submit-button', 'n_clicks'), State('tipo-radio', 'value'), State('nome-dropdown', 'value'), State('pollutant-dropdown', 'value'))
def update_time_series(n_clicks, tipo, nome, pollutant):
    if n_clicks == 0 or not nome or not pollutant:
        return px.line(title="Please select all inputs and press the button.").update_layout(template='plotly_white')
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {"var_comune": nome if tipo == "Comune" else None, "var_provincia": nome if tipo == "Provincia" else None, "var_pollutant": pollutant}
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)
    if not data:
        return px.line(title=f"No data found for {tipo} '{nome}' and pollutant '{pollutant}'.").update_layout(template='plotly_white')
    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'])
    df = df.sort_values('data')
    fig = px.line(df, x='data', y='valore', markers=True, title=f"Time series of {pollutant} in {tipo.lower()} {nome}", labels={'valore': 'Average value', 'data': 'Date'})
    return fig.update_layout(template='plotly_white')

@app.callback([Output('bar-chart-dv8', 'figure'), Output('map-dv8', 'figure')],
              Input('btn-dv8', 'n_clicks'),
              State('pollutant-dropdown-dv8', 'value'),
              State('start-date-picker', 'date'),
              State('end-date-picker', 'date'))
def update_dv8(n_clicks, pollutant, start_date, end_date):
    if n_clicks == 0 or not pollutant or not start_date or not end_date:
        return {}, {}
    df = get_dv8_data(pollutant, start_date, end_date)
    if df.empty:
        return {}, {}

    provinces_gdf = get_all_province_shapes()
    df['prov_upper'] = df['province'].str.upper().str.strip()
    provinces_gdf['prov_upper'] = provinces_gdf['nome_provincia'].str.upper().str.strip()
    merged = provinces_gdf.merge(df, on='prov_upper', how='left')

    media_globale = merged.loc[merged['average_value'].notnull(), 'average_value'].mean()

    def color_group(row):
        if pd.isnull(row['average_value']):
            return 0.0
        elif row['average_value'] < media_globale:
            return 0.25
        else:
            return 0.75

    merged['color_code'] = merged.apply(color_group, axis=1)

    bar_fig = px.bar(df, x='province', y='average_value',
                     title=f"Media {pollutant} per provincia",
                     labels={'average_value': f'Concentrazione media di {pollutant}', 'province': 'Provincia'},
                     color_discrete_sequence=['blue'])
    bar_fig.add_hline(y=media_globale, line_dash='dash', line_color='red',
                      annotation_text=f"Media regionale: {media_globale:.2f}",
                      annotation_position="top left")
    bar_fig.update_layout(template='plotly_white')

    geojson = json.loads(merged.to_json())

    colorscale = [
        [0.0, 'lightgray'],
        [0.2499, 'lightgray'],
        [0.25, 'lightblue'],
        [0.7499, 'lightblue'],
        [0.75, 'red'],
        [1.0, 'red']
    ]

    map_fig = go.Figure(go.Choroplethmapbox(
        geojson=geojson,
        locations=merged['nome_provincia'],
        z=merged['color_code'],
        featureidkey='properties.nome_provincia',
        colorscale=colorscale,
        marker_line_width=1,
        marker_line_color='black',
        zmin=0,
        zmax=1,
        showscale=False,            # togli la legenda a destra
        marker_opacity=0.6,         # aumenta la trasparenza
        hoverinfo='skip'            # togli il tooltip che mostra il valore
    ))

    map_fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=7,
        mapbox_center={"lat": 45.5, "lon": 9.0},
        margin={"r":0,"t":30,"l":0,"b":0},
    )

    return bar_fig, map_fig


if __name__ == '__main__':
    app.run(debug=True)
